## Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv.

Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок.

Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

### 1-ый вариант

In [1]:
import pandas as pd

In [2]:
# читаем исходный файл

df_rating = pd.read_csv('ratings.csv')
df_rating.tail()

,userId,movieId,rating,timestamp
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663
100003,671,6565,3.5,1074784724


In [3]:
# переводим время в "понятный" формат

df_rating['time'] = pd.to_datetime(df_rating['timestamp'], unit='s')
df_rating.tail()

,userId,movieId,rating,timestamp,time
99999,671,6268,2.5,1065579370,2003-10-08 02:16:10
100000,671,6269,4.0,1065149201,2003-10-03 02:46:41
100001,671,6365,4.0,1070940363,2003-12-09 03:26:03
100002,671,6385,2.5,1070979663,2003-12-09 14:21:03
100003,671,6565,3.5,1074784724,2004-01-22 15:18:44


In [4]:
# группируем данные по userId и определяем max значения
# переименовываем столбец и оставляем необходимые данные

time_max = df_rating.groupby('userId').max()
time_max.rename(columns={'time': 'max_time'}, inplace=True)
time_max = time_max[['max_time']].reset_index()
time_max.tail()

,userId,max_time
666,667,1996-11-06 09:23:54
667,668,2001-06-27 03:44:38
668,669,2002-03-11 06:56:06
669,670,1999-10-26 13:20:33
670,671,2004-01-22 15:18:55


In [5]:
# аналогично с min значениями

time_min = df_rating.groupby('userId').min()
time_min.rename(columns={'time': 'min_time'}, inplace=True)
time_min = time_min[['min_time']].reset_index()
time_min.tail()

,userId,min_time
666,667,1996-11-06 09:07:00
667,668,2001-06-27 03:39:56
668,669,2002-03-11 06:44:41
669,670,1999-10-01 12:35:28
670,671,2003-09-14 00:52:31


In [6]:
# объединяем две таблицы со временем
# рассчитываем время жизни для каждого userId

time = time_max.merge(time_min, how='left', on='userId')
time['time_life'] = time['max_time'] - time['min_time']
time.tail()

,userId,max_time,min_time,time_life
666,667,1996-11-06 09:23:54,1996-11-06 09:07:00,0 days 00:16:54
667,668,2001-06-27 03:44:38,2001-06-27 03:39:56,0 days 00:04:42
668,669,2002-03-11 06:56:06,2002-03-11 06:44:41,0 days 00:11:25
669,670,1999-10-26 13:20:33,1999-10-01 12:35:28,25 days 00:45:05
670,671,2004-01-22 15:18:55,2003-09-14 00:52:31,130 days 14:26:24


In [7]:
# группируем данные по userId и определяем количество (оценок)
# переименовываем столбец и оставляем необходимые данные

rating = df_rating.groupby('userId').count()
rating.rename(columns={'rating': 'quantity'}, inplace=True)
rating = rating[['quantity']].reset_index()
rating.tail()

,userId,quantity
666,667,68
667,668,20
668,669,37
669,670,31
670,671,115


In [8]:
# объединяем таблицы со временем жизни и количеством оценок
# оставляем необходимые для расчета данные

result = time.merge(rating, how='left', on='userId')
result = result[['time_life', 'quantity']]
result.tail()

,time_life,quantity
666,0 days 00:16:54,68
667,0 days 00:04:42,20
668,0 days 00:11:25,37
669,25 days 00:45:05,31
670,130 days 14:26:24,115


In [9]:
# рассчитываем среднее время жизни пользователей, поставивших больше 100 оценок

result_mean = result.loc[result['quantity'] > 100].mean()[['time_life']]
result_mean

time_life    463 days 21:28:27.449612
dtype: object

### 2-ой вариант

In [10]:
# задачу можно решить и за меньшее количество действий
# после перевода времени в "нормальный" вид, выполняем следующее:

df_agg = df_rating.groupby('userId').agg({'time': ['max', 'min'], 'rating': 'count'})
df_agg.tail()

time                     rating
                       max                 min  count
userId                                               
667    1996-11-06 09:23:54 1996-11-06 09:07:00     68
668    2001-06-27 03:44:38 2001-06-27 03:39:56     20
669    2002-03-11 06:56:06 2002-03-11 06:44:41     37
670    1999-10-26 13:20:33 1999-10-01 12:35:28     31
671    2004-01-22 15:18:55 2003-09-14 00:52:31    115

In [11]:
# определяем время жизни пользователей

df_agg['time_life'] = df_agg['time']['max'] - df_agg['time']['min']
df_agg.tail()

time                     rating         time_life
                       max                 min  count                  
userId                                                                 
667    1996-11-06 09:23:54 1996-11-06 09:07:00     68   0 days 00:16:54
668    2001-06-27 03:44:38 2001-06-27 03:39:56     20   0 days 00:04:42
669    2002-03-11 06:56:06 2002-03-11 06:44:41     37   0 days 00:11:25
670    1999-10-26 13:20:33 1999-10-01 12:35:28     31  25 days 00:45:05
671    2004-01-22 15:18:55 2003-09-14 00:52:31    115 130 days 14:26:24

In [12]:
# рассчитываем среднее время жизни пользователей, поставивших больше 100 оценок

df_agg.loc[df_agg['rating']['count'] > 100].mean()[['time_life']]

time_life      463 days 21:28:27.449612
dtype: object

## Задание 2

Дана статистика услуг перевозок клиентов компании по типам:

- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

Необходимо сформировать две таблицы:

1) таблицу с тремя типами выручки для каждого client_id без указания адреса клиента;

2) аналогичную таблицу по типам выручки с указанием адреса клиента.

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [13]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [14]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [15]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [16]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [17]:
# объединяем всё - это одна итоговая таблица

base = (client_base
        .merge(rzd, how='left', on='client_id')
        .merge(auto, how='left', on='client_id')
        .merge(air, how='left', on='client_id')
        ).fillna(0)
base

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0
2,113,Левобережная 1а,10283.0,57483.0,0.0
3,114,Мира 14,5774.0,83.0,0.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,0.0,4834.0,4.0
6,117,Панфиловская 33,0.0,98.0,13.0
7,118,Мастеркова 4,0.0,0.0,173.0


In [18]:
# вторая итоговая таблица

base_1 = base.drop(columns='address')
base_1

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


## Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы.

Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

1) У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?

2) Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?

3) Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

#### Ответ 

1) координаты (широта/долгота) могут быть записаны в виде: (55,755831°, 37,617673°), или (N55.755831°, E37.617673°), или (55°45.35′N, 37°37.06′E), или (55°45′20.9916″N, 37°37′3.6228″E). В любом случае, для получения одного-двух чисел можно использовать метод str.contains + соответствующее регулярное выражение (предварительно сделать import re).

2) пользователь может совершать покупки в любой точке земного шара и, соответственно, наборы координат будут по разному удалены друг от друга. Данный факт никак не отразится на расчетах пункта 1), просто для каждого user_id будет несколько пар "вариантов". Вопрос в формате представления всех исходных данных, а также в формулировке задачи - что конкретно требуется получить.    

3) координаты (широта/долгота) - это определенная точка на земной поверхности. Пользователь, как правило, совершает покупки в районе своего местожительства, или рядом с работой, или в супермаркетах по дороге на "приусадебное хозяйство". По частоте совершаемых покупок в том или ином месте, можно судить о предпочтениях пользователя, как по месту, так и по категории покупок. А уже на основе этой информации можно, например, формировать пул предложений по скидкам на эти и сопутствующие товары в конкретной географической точке (или рядом с ней), + делать это с учетом дней недели (рабочие / выходные) и т.д.

В целом, формат использования координат зависит от бизнес-задачи: надо ли получить распределение по странам, либо необходима группировка по городам, либо требуется более детальная информация по районам / улицам.